# `Postgresql Tips & Tricks:`

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

# Purpose & Outcome:

+ Psycopg2 goodies
+ Coalesce
+ find NA, NULL, empty strings
+ Duplicate Rows
+ Lag/Lead
+ Pandas Query sql
+ Timing query
+ Outliers

+ **Outcome**: cool skill building for your tool belt! and bragging

# Next Video: R-studio or Starting Machine Learning 🤯 😳

In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql,extras

# Import the 'config' function from the config_user_dta.py file:
from config_user_dta import config  # call my user credentials


`---------------------------------`

If you do not want to create init or config files: do something similar to this

**`import psycopg2
conn = psycopg2.connect("dbname=test user=postgres") # Connect to an existing database
cur = conn.cursor() #Open a cursor to perform database operations`**

In [2]:
# Establish a connection to the database by creating a cursor object

# Get the config params
params_ = config()

# Connect to the Postgres_DB:
conn = psycopg2.connect(**params_)

# Create new_cursor allowing us to write Python to execute PSQL:
cur = conn.cursor()

conn.autocommit = True  # read documentation understanding when to Use & NOT use (TRUE)

# `Misc. Goodies for Psycopg2` 🤠


**`-----------------------------------------`**

# `psycopg2.extras.DictCursor:`

+ retreive attributes in the a fashion similar to Python dictionaries



In [3]:
# usually I don't have anything inside this!
dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

dict_cur.execute("SELECT * FROM car_dealer_02;")

dd=dict_cur.fetchall() # save our query result variable

col_nms=[]
for i in dict_cur.description:
    col_nms.append(i[0]) # append list of column names for DF

pd.DataFrame(dd,columns=col_nms)

,publish_date,vin,make,model,model_year
0,2020-08-28,102280487,Kia,Soul,2019
1,2020-08-28,102298407,Chevrolet,Malibu,2017
2,2020-08-28,102633593,Ford,Fusion,2017


In [4]:
# What else can we do?

# if you know the key you want to get
for i in dd: 
    print('printing Vin: ',i['vin'])

# calling rows:
print('-------------')
print('printing row 02: ',dd[2])

printing Vin:  102280487
printing Vin:  102298407
printing Vin:  102633593
-------------
printing row 02:  [datetime.date(2020, 8, 28), 102633593, 'Ford', 'Fusion', 2017]


In [5]:
# can't access, like above! uncomment and see

# cur.execute('select * from car_dealer_02')
# d_=cur.fetchall()
# for i in d_:
#     print(i['vin']) # get an error!

# `psycopg2.extras.NamedTupleCursor:`

+ well you guessed it: makes us some tuples 

In [6]:
named_tup_cur = conn.cursor(cursor_factory=psycopg2.extras.NamedTupleCursor)
named_tup_cur.execute("select * from car_dealer_01")

# returns a psycopg2.extras.Record stored in a list of Named Tuples NOT regular tuples
nn=named_tup_cur.fetchall()

# get our trusty old column names: yee haw
named_tup_cur.description

# to fetch from Named Tuples: use either index or dot notation by name:

print('By index (row=0): ',nn[0])
print('----------------')
print('Or by dot notation for (row=0) return entry for vin:', nn[0].vin)

By index (row=0):  Record(vin='102280487', idx=0, model_number_list="['B2522']")
----------------
Or by dot notation for (row=0) return entry for vin: 102280487


In [7]:
# We can directly take this for a Dataframe by the way: yay
pd.DataFrame(nn).head()

,vin,idx,model_number_list
0,102280487,0,['B2522']
1,102280487,0,['B2522']
2,102298407,1,['1ZD69']
3,102298407,1,['1ZD69']
4,102298407,1,['1ZD69']


In [8]:
def create_staging_table(cursor):
    cursor.execute("""
        DROP TABLE IF EXISTS null_table_ex;
        CREATE UNLOGGED TABLE null_table_ex (
            first_name         varchar(30),
            bank_balance       int,
            personal_info      text
            
        );""")
    
# Send the Schema to PSQL

with conn.cursor() as cursor:
    create_staging_table(cursor)

In [9]:
q_="""
insert into null_table_ex values
('john',10,'something interesting'),
('sue',200,'unknown'),
('larry',NULL,'average place'),
('queen troll',NULL,'trollville')
"""
cur.execute(q_)

In [10]:
cur.execute('select * from null_table_ex')
cur.fetchall()

[('john', 10, 'something interesting'),
 ('sue', 200, 'unknown'),
 ('larry', None, 'average place'),
 ('queen troll', None, 'trollville')]

# `Coalesce:` returns first argument that is not Null

+ you can take advantage of this by recoding null values to whatever filler word you want, like `fillna()` in python  

In [11]:
coalesce_="""
SELECT 
    *,
    COALESCE(bank_balance, -1) AS RECODED_NULL_VAL
FROM  
  null_table_ex    
"""
cur.execute(coalesce_)
coale_cols=[]
for i in cur.description:
    coale_cols.append(i[0])

pd.DataFrame(cur.fetchall(),columns=coale_cols)

# side note watch the datatype, the column of bank_balance gives 
# me issues if I don't pay attn

,first_name,bank_balance,personal_info,recoded_null_val
0,john,10.0,something interesting,10
1,sue,200.0,unknown,200
2,larry,NaN,average place,-1
3,queen troll,NaN,trollville,-1


# `Prints off: Nulls rows`

In [12]:

cur.execute("""
SELECT t.*
FROM null_table_ex as t

-- If nulls are present, these will not be equal

WHERE to_jsonb(t) != jsonb_strip_nulls(to_jsonb(t))
""")
cur.fetchall()

# not my code, got from stackoverflow, citation at bottom

[('larry', None, 'average place'), ('queen troll', None, 'trollville')]

# `Check if you have an empty string or Null:`  Case When

+ you are unable to use `coalesce` for this circumstance... Sad face, but wait super-heroes let's use **`case when`** and keep the *mega trolls* at bay.
    + think of `Case When` as a conditional that keeps going until it finds a true value


`SIDE Note:`

**In the terminal**: `\pset`

ex.) `\pset null (null)` then run your query and it should fill with `(null)`

check this link for cool tricks/tips in command line

https://pgdash.io/blog/postgres-psql-tips-tricks.html

In [13]:
def fun_table_creation(table:str,col_01:str,col_02:str,col_03:str,col_04:str):
    with conn.cursor() as cursor:
        stmnt=sql.SQL("""
        DROP TABLE IF EXISTS {table};
        CREATE UNLOGGED TABLE {table} (
            {col_01}        varchar(30),
            {col_02}        int,
            {col_03}        text,
            {col_04}        varchar(12)
        
        );""").format(table=sql.Identifier(table),col_01=sql.Identifier(col_01),
col_02=sql.Identifier(col_02),col_03=sql.Identifier(col_03),col_04=sql.Identifier(col_04))
        result= cur.execute(stmnt)
        
#     result = cur.fetchall() # this can't be used when created the table
    return result
    
fun_table_creation('table_null_empty_str','first_name','bank_balance',
                  'personal_info','phone_number')

# `Check Our Table and Schema:`

In [14]:
ss="""
SELECT table_name, 
column_name, 
data_type 
FROM information_schema.columns 
WHERE table_name={table_name} ;
"""

# we are using: literal because the table_name will call our table and we need str repr.
o=sql.SQL(ss).format(table_name=sql.Literal('table_null_empty_str'))
cur.execute(o)
cur.fetchall()

[('table_null_empty_str', 'first_name', 'character varying'),
 ('table_null_empty_str', 'bank_balance', 'integer'),
 ('table_null_empty_str', 'personal_info', 'text'),
 ('table_null_empty_str', 'phone_number', 'character varying')]

# `Insert into table: empty strings, null etc`



In [15]:
cur.execute("insert into table_null_empty_str values\
('George',NULL,'vagabond',''),\
('Spencer',20,'saturn','9999999999'),\
('August',100,'who knows','1111111111'),\
('Victor',NULL,'NA','')\
")


In [16]:
cur.execute("select * from table_null_empty_str")
cur.fetchall()

[('George', None, 'vagabond', ''),
 ('Spencer', 20, 'saturn', '9999999999'),
 ('August', 100, 'who knows', '1111111111'),
 ('Victor', None, 'NA', '')]

# Find the empty strings, null and NA values:

In [17]:
check_emptyStr_="""
SELECT 
 *,
COALESCE(bank_balance, -1) AS RECODED_NULL_VAL_Bank_balance, 
-- same as befor with bank_balance

CASE WHEN phone_number = '' THEN 'EMPTY_MISSING' END AS CASEWHEN_EMPTY_STR_Phone, 
-- check if phone_num col has empty strings and replace with empty_missing str

CASE WHEN personal_info = 'NA' THEN 'NA_MISSING' END AS CASEWHEN_NA_STR_Personal_Info
--same as above checks but with personal_info col, where NA is a string NA

FROM 
   table_null_empty_str
"""

cur.execute(check_emptyStr_)
co_=[]
for i in cur.description:
    co_.append(i[0])
    
    
pd.DataFrame(cur.fetchall(),columns=co_)

,first_name,bank_balance,personal_info,phone_number,recoded_null_val_bank_balance,casewhen_empty_str_phone,casewhen_na_str_personal_info
0,George,NaN,vagabond,,-1,EMPTY_MISSING,None
1,Spencer,20.0,saturn,9999999999,20,None,None
2,August,100.0,who knows,1111111111,100,None,None
3,Victor,NaN,NA,,-1,EMPTY_MISSING,NA_MISSING


# `Find Duplicate Rows: `

In [18]:

cur.execute("""SELECT (ppl_cpu_purchases.*)::text, count(*)
FROM ppl_cpu_purchases
GROUP BY ppl_cpu_purchases.*
HAVING count(*) > 1""")

print('Duplicate Value: ',cur.fetchall())

print('--------------')

# verify
cur.execute("""select * from ppl_cpu_purchases where last_name='DataScience' """)
print('Verify Duplicate: ',cur.fetchall())

Duplicate Value:  [('(9999-9999-9999-9999,MrFugu@gmail.com,MrFuguYay,DataScience,1111111111,"Intel Core i9-9999K",2020-05-05)', 2)]
--------------
Verify Duplicate:  [('9999-9999-9999-9999', 'MrFugu@gmail.com', 'MrFuguYay', 'DataScience', '1111111111', 'Intel Core i9-9999K', datetime.date(2020, 5, 5)), ('9999-9999-9999-9999', 'MrFugu@gmail.com', 'MrFuguYay', 'DataScience', '1111111111', 'Intel Core i9-9999K', datetime.date(2020, 5, 5))]


# `Lag( ) and Lead( )`: allowing you to move forward or backward

+ nonaggregate window function (perform operations by row) and return it  versus an aggregate function that returns a single row

+ You can use these if you are searching through dates for instance. 

# `Ex. )` 

+ What if I wanted to find a column relating to a particular cpu version and when it was last sold

*Also, better than doing self-join*: https://cloud.google.com/bigquery/docs/best-practices-performance-patterns

In [19]:
# Lag: 

lag_v="""
SELECT first_name, last_name, purchase_date, cpu, 
lag(purchase_date) OVER (
PARTITION BY cpu
ORDER BY purchase_date) lag_date_col
FROM ppl_cpu_purchases"""
# cur.execute('select * from ppl_cpu_purchases limit 2')
cur.execute(lag_v)
lag_cols=[]
for i in cur.description:
    lag_cols.append(i[0])
pd.DataFrame(cur.fetchall(),columns=lag_cols).head(10)
# cur.mogrify(u)

,first_name,last_name,purchase_date,cpu,lag_date_col
0,Ebany,Rivera,2017-07-10,AMD Ryzen 1 1113X,None
1,Terri,Fadlelmola,2018-08-10,AMD Ryzen 1 1113X,2017-07-10
2,Austin,al-Abdo,2018-08-28,AMD Ryzen 1 1140X,None
3,Brooke,Ly,2017-12-25,AMD Ryzen 1 1147X,None
4,David,el-Rabbani,2018-06-27,AMD Ryzen 1 1157X,None
5,Rashaa,Redwood,2017-12-31,AMD Ryzen 1 1183X,None
6,Mario,el-Kazmi,2018-02-27,AMD Ryzen 1 1208X,None
7,Leshai,Green,2017-10-22,AMD Ryzen 1 1215X,None
8,Justin,Becerra,2018-03-06,AMD Ryzen 1 1288X,None
9,Shaahira,Le,2018-07-18,AMD Ryzen 1 1383X,None


 **Avoiding the use of a self join will aid in performance for big queries and the potential of duplicating output. Using a window function will assist in your row dependent relationships**
 
`-------------------------------`

# Sql query with python:

we can directly use pandas here:

`query="Some sql query"
pd.read_sql(query,connection)
`

but there is a problem here; if we need to do constant updates to these data we have to think about this. Also, if the query is returning a lot of data we would like to instead consider bringing the data into python. 

In [20]:
query="""select * from car_dealer_01"""
pd.read_sql(query,conn).head(6)

# Don't use this if there are frequent updates to your data or have large files. Then c
# consider writing a function or usual query protocol with your connection

,vin,idx,model_number_list
0,102280487,0,['B2522']
1,102280487,0,['B2522']
2,102298407,1,['1ZD69']
3,102298407,1,['1ZD69']
4,102298407,1,['1ZD69']
5,102298407,1,['1ZD69']


`------------------------------`


# `Timing our queries:`

`--------------------------`

**using the terminal you have**: `\timing`

which can be used like this:

**`my terminal_name# \timing
my_terminal_name# select * from some_table;
`**

`you will get a printout: But, what if I am using python?`

`---------------------------`

**`explain analyze`** can help

we can have a query like this:

**`explain analyze select * from some_table`**

you will get some data printed out:

`QUERY PLAN`

`------------------------------------------------------`

**`Seq Scan on ppl_cpu_purchases  (cost=0.00..143.26 rows=5826 width=85) (actual 
 time=0.019..1.413 rows=5828 loops=1)
 Planning Time: 0.058 ms
 Execution Time: 1.904 ms`**

`-------------------------------`

# `Find Outliers: ` (without) using self-join

+ For instance we are trying to find the max/min value for each row based on a particular ID value.


"""SELECT *
FROM 
(
  SELECT 
    DAT.*, 
    CASE WHEN (NUM_VAR = MAX(NUM_VAR) OVER (PARTITION BY ID_VAR)) THEN 'Y' ELSE 'N' END AS MAX_NUM_IND
  FROM 
    CURRENT_TABLE     DAT
) DAT2
WHERE MAX_NUM_IND = 'Y'"""

`Not my code`

# <font color=red>LIKE</font>, Share &

# <font color=red>SUB</font>scribe

# Citaitons & Help:

# ◔̯◔

`Misc. psycopg2 goodies`:

https://www.psycopg.org/docs/extras.html?highlight=loggingconnection

https://stackoverflow.com/questions/6739355/dictcursor-doesnt-seem-to-work-under-psycopg2

https://stackoverflow.com/questions/49020718/get-a-list-of-column-names-from-a-psycopg2-cursor-executing-stored-proc

`Coalesce & Nulls`

https://stackoverflow.com/questions/59458925/postgres-get-all-columns-that-have-no-null-values

https://stackoverflow.com/questions/31034847/how-to-find-all-rows-with-a-null-value-in-any-column-using-postgresql

https://towardsdatascience.com/6-sql-tricks-every-data-scientist-should-know-f84be499aea5

https://docs.yugabyte.com/latest/api/ysql/datatypes/type_json/functions-operators/jsonb-strip-nulls/ (jsonb not null functions)

https://www.postgresql.org/docs/9.5/functions-json.html (json functions)

`Stats`


https://www.postgresql.org/docs/9.6/monitoring-stats.html

https://gist.github.com/anvk/475c22cbca1edc5ce94546c871460fdd

`Tricks`

https://towardsdatascience.com/6-sql-tricks-every-data-scientist-should-know-f84be499aea5

https://pgdash.io/blog/postgres-psql-tips-tricks.html

https://www.linkedin.com/pulse/advanced-sql-queries-tips-tricks-arash-atarzadeh/

https://medium.com/better-programming/4-sql-tips-for-data-scientist-and-data-engineers-56c41487752f

https://www.mysqltutorial.org/mysql-window-functions/mysql-lag-function/ (lag/lead)

https://www.oracletutorial.com/oracle-analytic-functions/oracle-lead/    (lag/lead)

https://stackoverflow.com/questions/28156795/how-to-find-duplicate-records-in-postgresql (dups)